In [8]:
import xgboost as xgb
import dask.array as da
import dask.distributed

cluster = dask.distributed.LocalCluster(n_workers=90, threads_per_worker=1,diagnostics_port=None,processes=True)
client = dask.distributed.Client(cluster)

# X and y must be Dask dataframes or arrays
num_obs = 1e5
num_features = 20
X = da.random.random(
    size=(num_obs, num_features),
    chunks=(1000, num_features)
)
y = da.random.random(
    size=(num_obs, 1),
    chunks=(1000, 1)
)

dtrain = xgb.dask.DaskDMatrix(client, X, y)

output = xgb.dask.train(client,
                        {'verbosity': 2,
                         'tree_method': 'hist',
                         'objective': 'reg:squarederror'
                         },
                        dtrain,
                        num_boost_round=4, evals=[(dtrain, 'train')])

/apps/pangeo/2020.05/lib/python3.7/site-packages/distributed/node.py:244: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44693 instead
  http_address["port"], self.http_server.port


In [9]:
output

{'booster': <xgboost.core.Booster at 0x1476814ed0d0>,
 'history': {'train': OrderedDict([('rmse',
                [0.28774, 0.287408, 0.2870400000000001, 0.286745])])}}

In [7]:
client.shutdown()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting

In [16]:
import numpy as np
import pandas as pd
import dask.dataframe as dd


df_test = dd.read_csv('input_dataset_filtered.csv')

float_cols = [c for c in df_test if df_test[c].dtype == "float64"]
float32_cols = {c: np.float32 for c in float_cols}

df =  dd.read_csv('input_dataset_filtered.csv', engine='c', dtype=float32_cols)

X, y = df.iloc[:, 1:], df.iloc[:, :1]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

NameError: name 'train_test_split' is not defined

In [ ]:
from dask.distributed import worker_client
from dask.distributed import Client,LocalCluster
client = Client(scheduler_file='scheduler.json')
client

In [ ]:
from dask_ml.model_selection import GridSearchCV, train_test_split
import xgboost as xgb
# from dask_ml.xgboost import XGBRegressor
import dask.dataframe as df


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)

# A parameter grid for XGBoost
# parameters = {
#     'min_child_weight':[4,5], 
#     'gamma':[i/10.0 for i in range(3,6)],  
#     'subsample':[i/10.0 for i in range(6,11)],
#     'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]
# }
parameters = {
    'min_child_weight':[4], 
    'gamma':[3/10.0],  
    'subsample':[6/10.0],
    'colsample_bytree':[6/10.0], 
    'max_depth': [2]
}

xgb_model = xgb.XGBRegressor()
clf = GridSearchCV(
    xgb_model, 
    parameters,
    cv = 2,
    n_jobs = -1,
    verbose=True
)
clf
# clf.fit(X_train, y_train)  # doctest: +NORMALIZE_WHITESPACE +ELLIPSIS
# sorted(clf.cv_results_.keys())  # doctest: +NORMALIZE_WHITESPACE +ELLIPSIS


In [13]:
clf.cv_results_

{'params': [{'C': 1, 'kernel': 'linear'},
  {'C': 1, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'}],
 'mean_fit_time': array([0.00142386, 0.0010854 , 0.0007633 , 0.00074151]),
 'std_fit_time': array([2.04212801e-04, 2.58847092e-04, 1.79149584e-04, 6.97394782e-05]),
 'mean_score_time': array([0.0009462 , 0.0005448 , 0.0003221 , 0.00045657]),
 'std_score_time': array([1.27455935e-04, 1.10789033e-04, 1.97388266e-05, 1.28511531e-04]),
 'split0_test_score': array([1.  , 0.96, 1.  , 0.98]),
 'split1_test_score': array([1.  , 0.98, 0.94, 1.  ]),
 'split2_test_score': array([0.98, 0.94, 0.98, 0.94]),
 'mean_test_score': array([0.99333333, 0.96      , 0.97333333, 0.97333333]),
 'std_test_score': array([0.00942809, 0.01632993, 0.02494438, 0.02494438]),
 'rank_test_score': array([1, 4, 2, 2], dtype=int32),
 'param_C': masked_array(data=[1, 1, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kern